# Monthly wages for females and males
In LOCAL CURRENCY UNITS

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import json
import csv
from collections import defaultdict
import functools
import operator

In [2]:
# Read csv files from No Ceilings project folder
path = "/Users/ericaxia/Downloads/Github/project-girlboss/data/noceilings-data-master/csv"
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
# Narrow down to just the files I want
files = ['MONWAGFE.csv', 'MONWAGMA.csv']
dfs = []
for f in files:
    df = pd.read_csv(f)
    dfs.append((f, df))    

## Let's narrow down to a chosen year

In [3]:
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/d3layout_data")

In [4]:
## NEW: map ISO -> Country name for better understandability in graph
with open('iso_to_country_names.csv', mode='r') as infile:
    reader = csv.reader(infile)
    country_names = {rows[0]: rows[1] for rows in reader}
    # print(country_names)

country_names_dict = { v: k for k, v in country_names.items()}
# country_names_dict


In [5]:
def convert_iso_to_name(iso):
    # print(x)
    if iso in country_names_dict:
        return country_names_dict[iso]
    else:
        return iso

In [28]:
wages_fem = dfs[0][1]  # monthly wages for females
wages_m = dfs[1][1] # monthly wages for males


In [29]:
wages_fem = wages_fem.iloc[0:85, :]  # rm bottom descrip rows
wages_m = wages_m.iloc[0:85, :]  # rm bottom descrip rows

In [30]:
## Missing values by year
# wages_fem.isna().sum().sort_values()[0:5]

In [31]:
# narrow down to countries that DO have data for 2011
wages_fem = wages_fem[~wages_fem['2011'].isna()]
wages_m = wages_m[~wages_m['2011'].isna()]
print(wages_fem.shape, wages_m.shape) # (26,18) -> we have 26 countries

(26, 18) (26, 18)


In [32]:
# TODO: clean dfs, and export to JSON format needed for bar charts
wages_fem = wages_fem.fillna(axis=0, method='pad')
wages_m = wages_m.fillna(axis=0, method='pad')

In [33]:
# wages_fem['gender'] = 'female'
# wages_m['gender'] = 'male'


In [34]:
# wages_fem.isna().sum().sum()
# wages_m.isna().sum().sum()

0

In [61]:
wages_fem['ISO'].unique()

array(['AUS', 'CAN', 'CZE', 'ECU', 'DEU', 'GTM', 'ISL', 'LVA', 'LUX',
       'MLT', 'MEX', 'MNG', 'NZL', 'NOR', 'PAK', 'PHL', 'QAT', 'SGP',
       'SVK', 'ESP', 'SWE', 'SYR', 'THA', 'UKR', 'GBR', 'URY'],
      dtype=object)

In [74]:
c = 'AUS'
f1 = wages_fem[wages_fem['ISO'] == c]
m1 = wages_m[wages_m['ISO'] == c]
test = pd.concat([f1, m1], axis=0)
test.head()

,ISO,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
2,AUS,1867.23,1923.89,1996.15,2043.60,2081.95,2187.25,2286.27,2347.15,2450.61,2552.77,2680.17,2790.67,2931.82,3047.20,3181.43,3341.43,3473.27
2,AUS,2846.68,2924.68,3013.51,3116.64,3172.43,3299.18,3439.15,3598.51,3790.26,3893.07,4102.37,4289.13,4502.88,4689.64,4903.38,5152.77,5373.88


In [75]:
f2 = f1.melt().iloc[1:, :]
f2.columns = ['year', 'female']
m2 = m1.melt().iloc[1:, :]
m2.columns = ['year', 'male']
all = f2.merge(m2, on='year')
# all['avg'] = all['female'] + all['male']
# all['avg'] = all['avg'] /2 
all.head()

,year,female,male
0,1995,1867.23,2846.68
1,1996,1923.89,2924.68
2,1997,1996.15,3013.51
3,1998,2043.6,3116.64
4,1999,2081.95,3172.43


In [76]:
fname = "bar_test2.csv"
all.to_csv("bar_test2.csv", index=False)